In [1]:
# 1. 랭스미스 초기화

import os
from dotenv import load_dotenv
from langchain.callbacks.manager import tracing_v2_enabled

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_PROJECT"] = "llm_compiler_KB"

# 트레이싱 상태 확인
with tracing_v2_enabled() as tracing:
    print(f"Tracing enabled: {tracing}")


Tracing enabled: <langchain_core.tracers.langchain.LangChainTracer object at 0x1168d8410>


In [2]:
from tools.webSearch.webSearch_tool import WebSearchTools
from langchain_openai import ChatOpenAI

llm_mini = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

web_search_tool = WebSearchTools(llm_mini)

web_search_tool.invoke({"query":"최근 영업이익 알려줘", "company":"SK스퀘어"})

검색 쿼리: SK스퀘어 최근 영업이익 알려줘
google_results: [{'tool': '검색 도구[구글]', 'title': '[단독] SK CEO세미나서 리밸런싱 성적 점검...장녀 최윤정 참석 | 중앙 ...', 'link': 'https://www.joongang.co.kr/article/25288215', 'referenced_content': '2024. 10. 29. ... 또한 세미나에서는 3분기에 역대 최대인 7조 원대 영업이익을 기록한 SK하이닉스의 경영 성과가 스포트라이트를 받을 것으로 보인다. 그룹 차원에서\xa0...'}, {'tool': '검색 도구[구글]', 'title': '11번가, 사회연대은행과 소상공인 셀러 20억 무이자 대출 지원', 'link': 'https://www.digitaltoday.co.kr/news/articleView.html?idxno=538450', 'referenced_content': '2024. 10. 24. ... "인공지능 MD가 상품 정보 알려줘" · 11 ... 11번가, 서울시 손잡고 소상공인 판로 확대 · 11번가, 오픈마켓 부문 6개월 연속 영업이익 흑자 · SK스퀘어\xa0...'}, {'tool': '검색 도구[구글]', 'title': '유한양행 - 나무위키', 'link': 'https://namu.wiki/w/%EC%9C%A0%ED%95%9C%EC%96%91%ED%96%89', 'referenced_content': '3일 전 ... SK스퀘어 로고.svg. SK스퀘어. 41. 카카오뱅크 로고.svg 카카오뱅크. 42 ... 영업 이익. 360억 2,900만 원 (2022). 순이익. 905억 9,300만 원 (2022).'}, {'tool': '검색 도구[구글]', 'title': 'ArcheoBiblioBase: Archives in Russia', 'link': 'https://abb.eastview.com/rep/D-20.tab3.php?b', 'referenced_c

{'output': '[[단독] SK CEO세미나서 리밸런싱 성적 점검...장녀 최윤정 참석 | 중앙 ...] 2024. 10. 29. ... 또한 세미나에서는 3분기에 역대 최대인 7조 원대 영업이익을 기록한 SK하이닉스의 경영 성과가 스포트라이트를 받을 것으로 보인다. 그룹 차원에서\xa0...\n\n[11번가, 사회연대은행과 소상공인 셀러 20억 무이자 대출 지원] 2024. 10. 24. ... "인공지능 MD가 상품 정보 알려줘" · 11 ... 11번가, 서울시 손잡고 소상공인 판로 확대 · 11번가, 오픈마켓 부문 6개월 연속 영업이익 흑자 · SK스퀘어\xa0...\n\n[유한양행 - 나무위키] 3일 전 ... SK스퀘어 로고.svg. SK스퀘어. 41. 카카오뱅크 로고.svg 카카오뱅크. 42 ... 영업 이익. 360억 2,900만 원 (2022). 순이익. 905억 9,300만 원 (2022).\n\n[ArcheoBiblioBase: Archives in Russia] 2020. 12. 7. ... Tsentral\'nyi gosudarstvennyi arkhiv nauchno-tekhnicheskoi dokumentatsii Sankt-Peterburga (TsGANTD SPb). Holdings Total: 504 fonds, 521,300 units\xa0...\n\n[[특징주] 나노엔텍, 위고비 노보디스크 SK 북미공장 인수 추진 비만 ...] 2024. 10. 10. ... 나노엔텍은 한때 SK그룹 계열사로 있던 지난 2022년 SK스퀘어 ... 특히 부실채권 매각익을 제외한 충당금반영전영업이익(PPOP) 적자폭이 약\xa0...\n\n[SK스퀘어, 2분기 영업이익 7천748억원 - ZDNet korea] SK스퀘어는 2024년 2분기 연결재무제표 기준 매출 4천686억원, 영업이익 7천748억원, 순이익 7천288억원을 기록했다고 13일 밝혔다. SK하이닉스 등 포트폴리오 회사의 실적 개선으로 지분법 손익 총 8천247억원이...

In [2]:
import os
from tools.extractor.query_processor_tool import get_query_processor_tool
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatClovaX


llm_mini = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))
llm_clova = ChatClovaX(model="HCX-003", clovastudio_api_key=os.getenv("CLOVA_API_KEY"), temperature=0.1)

query_processor_tool = get_query_processor_tool(llm_mini, llm_clova)

query_processor_tool.invoke({"query":"네이버 최근 영업이익 알려줘"})

캐시된 순매수 가중치 사용
기존 기업 벡터스토어를 로드합니다...

=== 기업명 추출 시작 ===
입력 쿼리: 네이버 최근 영업이익 알려줘

[Step 1: LLM으로 회사명 텍스트 추출 중...]
LLM 응답: content='["네이버"]' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 317, 'total_tokens': 322, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_bd83329f63', 'finish_reason': 'stop', 'logprobs': None} id='run-fdc7939c-f069-4841-bc92-22240b0ec44c-0' usage_metadata={'input_tokens': 317, 'output_tokens': 5, 'total_tokens': 322, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
- LLM이 예상한 회사명 목록: ['네이버']

[Step 2: 네이버에 대한 유사 기업명 후보 추출 중...]
[('굿네이버', 0.75), ('네이버제트', 0.7), ('네이버핸즈', 0.7), ('네이버랩스', 0.7), ('네이버웹툰', 0.7), ('네이블', 0.6666666

HTTPStatusError: Error response 400 while fetching https://clovastudio.stream.ntruss.com/testapp/v1/chat-completions/HCX-003: {"status":{"code":"40080","message":"Model not found"},"result":null}

In [7]:
from deep_translator import GoogleTranslator

text = "네이버"
translated = GoogleTranslator(source='ko', target='en').translate(text)
print(translated)  # "Naver"

translated=translated.replace(" ","")
print(translated)

Naver
Naver


In [3]:
from langchain_core.prompts import ChatPromptTemplate

candidate_selection_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 주어진 기업명과 가장 일치하거나 유사한 회사를 후보 목록에서 찾는 전문가입니다.

주어진 후보 목록에서 다음 우선순위로 가장 적절한 회사를 하나만 선택하십시오:
1순위: 입력된 기업명과 목록의 회사명이 정확히 일치하는 경우
- 다음은 모두 동일한 회사로 간주하고 목록에 있는 형태를 반환합니다:
  * "네이버", "NAVER", "naver", "Naver" → 목록에 있는 "NAVER" 반환
  * "삼성", "SAMSUNG", "samsung" → 목록에 있는 형태로 반환
- 띄어쓰기는 무시합니다
2순위: 정확히 일치하는 것이 없는 경우, 입력된 기업명과 가장 유사한 회사

후보 회사 목록:
{candidates}

입력된 기업명: {query}

### 응답 규칙
1. 입력된 기업명이 후보 목록에 정확히 일치하는 것이 있다면 반드시 해당 회사를 선택합니다.
   예: 입력된 기업명이 "카카오"이고 목록에 ["카카오", "카카오뱅크"]가 있다면 반드시 "카카오"를 선택
2. 정확히 일치하는 회사가 없을 때만 가장 유사한 회사를 선택합니다.
3. 회사명만 목록에 있는 그대로 정확히 반환해야 합니다.
4. None은 절대 반환하지 않습니다.
5. 어떠한 설명이나 추가 텍스트도 포함하지 마십시오.
6. 따옴표나 기타 구두점 없이 회사명만 정확히 반환하십시오.
7. 출력은 회사명만 반환해야 합니다.

중요: 입력값과 목록의 회사명을 비교할 때는 반드시 대소문자와 한글/영문 표기를 동일하게 취급하여 비교하십시오.
    """),
])

list = ['네이버커넥트', '네이버웹툰', '네이버시스템', '네이버제트', '네이버랩스', '네이버파이낸셜', 'NAVER', '네이버핸즈', '네이버클라우드', '네이버아이앤에스', '굿네이버', '네이버웹툰컴퍼니', '바이버', '네이블', '이범', '해피네이버', '네이쳐글로벌', '화이버텍', '포네이처', '피네이션', '파이버로', '웨이버스', '바이버리', '더네이쳐', '아이리버', '네이처셀', '네이션스', '화이버랩', '네오버스', '사이버문', '네오실버', '씨네이천', '올네이션', '투네이션', '사이버넷', '네이처텍', '싸이버원', '네이처풀', '화이버넷', '금하네이벌텍', '넥싸이트', '마인버스', '대일버스', '십일번가', '넷게임즈', '뉴시스', '네오베이스', '네브라이트', '우성네이처', '사이버다임', '네이처스팜', '네모이엔지', '아이클로버', '청우네이처', '네오이뮨텍', '이레네이처', '에이원비버', '헬로네이처', '사이버패스', '제이엠실버', '네이처가든', '싸이버테크', '사이버메드', '하이네이처', '기가사이버', '에이원실버', '퓨처네이처', '사이버랜드', '바이오버드', '파이버프로']

clova_response = llm_clova.invoke(
                    candidate_selection_prompt.format(
                        query="s",
                        candidates=list
                    )
                )
print(clova_response)

content='NAVER' additional_kwargs={} response_metadata={'stop_reason': 'stop_before', 'input_length': 690, 'output_length': 4, 'seed': 3661009725, 'ai_filter': None} id='run-30cc416b-62fe-4051-b4f8-e78ce96e60bf-0' usage_metadata={'input_tokens': 690, 'output_tokens': 4, 'total_tokens': 694}


In [2]:
# 2. 체인 초기화 및 실행
from llm_compiler import initialize_chain
from langchain_core.messages import HumanMessage
from pprint import pprint

chain = initialize_chain()

# 3. 스트리밍 실행
for step in chain.stream(
    {
    "messages": [HumanMessage(content= "네이버 최근 영업이익 알려줘")],
    "key_information": []
    }
):
    print("step 디버깅 : ", step)

print("---------------------최종 답변 --------------------------")
pprint(step["join"]["messages"][-1].content)
print("\n---------------------참고 문헌 --------------------------")
if "key_information" in step["join"]:
    key_info = step["join"]["key_information"]
    if key_info:
        for info in key_info:
            print(f"\n📚 도구: {info.get('tool', 'N/A')}")
            print(f"📄 참고 내용: {info.get('referenced_content', 'N/A')}")
            if info.get('filename'):
                print(f"📎 파일명: {info.get('filename')}")
            if info.get('page_number'):
                print(f"📑 페이지: {info.get('page_number')}")
            if info.get('link'):
                print(f"🔗 링크: {info.get('link')}")
            if info.get('title'):
                print(f"📑 제목: {info.get('title')}")
            if info.get('broker'):
                print(f"🏢 증권사: {info.get('broker')}")
            if info.get('target_price'):
                print(f"💰 목표가: {info.get('target_price')}")
            if info.get('investment_opinion'):
                print(f"💡 투자의견: {info.get('investment_opinion')}")
            print("-" * 50)
    else:
        print("참고 문헌이 없습니다.")
print("------------------------------------------------------")

캐시된 순매수 가중치 사용
기존 벡터스토어를 로드합니다...
기존 기업 벡터스토어를 로드합니다...

=== 기업명 추출 시작 ===
입력 쿼리: 네이버 최근 영업이익 알려줘

[Step 1: LLM으로 회사명 텍스트 추출 중...]
LLM 응답: content='["네이버"]' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 459, 'total_tokens': 466, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-b679391d-bf5d-474b-89ee-24bfee9e919c-0' usage_metadata={'input_tokens': 459, 'output_tokens': 7, 'total_tokens': 466, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
- LLM이 예상한 회사명 목록: ['네이버']

[Step 2: 네이버에 대한 유사 기업명 후보 추출 중...]
{'굿네이버': 0.75, '네이버랩스': 0.7, '네이버핸즈': 0.7, '네이버웹툰': 0.7, '네이버제트': 0.7, '네이블': 0.66666666666666

ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)


(resolved 전)*args check *************
{'query': '네이버커넥트 영업이익', 'company': '네이버커넥트', 'year': 2023}

Resolving dict args: {'query': '네이버커넥트 영업이익', 'company': '네이버커넥트', 'year': 2023}
******************************
*Whatis my arg ? *************
네이버커넥트 영업이익
*Observations check *************
{}
******************************
*Whatis my arg ? *************
네이버커넥트
*Observations check *************
{}
******************************
*Whatis my arg ? *************
2023
*Observations check *************
{}
tool에 진짜 들어가는 형태의 args
{'query': '네이버커넥트 영업이익', 'company': '네이버커넥트', 'year': 2023}
observations {}
tool에 진짜 들어가는 형태의 args
{'query': '네이버커넥트 영업이익', 'company': '네이버커넥트', 'year': 2023}
입력 회사명 : 네이버커넥트, 입력 연도 : 2023
reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습니다.'}

reprt_code='11011', fs_div='CFS' (사업보고서, 연결제무제표)'
{'status': '013', 'message': '조회된 데이타가 없습

ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)
ERROR:langsmith._internal._serde:Failed to use model_dump to serialize <class 'tools.retrieve.financialReport.report_statement_tool.CombinedFinancialReportSearchTool'> to JSON: PydanticSerializationError(Unable to serialize unknown type: <class 'pydantic._internal._model_construction.ModelMetaclass'>)


KeyboardInterrupt: 